In [ ]:
#Black and White Numbers Binocular Rivaly Experiment Day 1

#DAY 1: Assess opacities of white and black numbers needed for this subject to see each color w/in a 40/60 time ratio
#     Subjects will be wearing binocular rivalry goggles which cause them to see each colored number in the same visual space
#     Subjects will be instructed to click the screen (to activate the window) when beginning the experiment
#     Subjects will be instructed to hit the 'b' key when they are seeing the black number and the 'w' key when they see white

#Program overview:
#     Expt adjusts opacities of B/W numbers on a gray background
#     Must be withing a certain standard (60-40 now) to pass and must do this for 2 trials in a row for BOTH color arrangements
#     Prints the opacities to achieve this if white is on the right or left and if black in on the left or right

from psychopy import visual, event, core
from psychopy.hardware import keyboard
import numpy as np

win = visual.Window(fullscr=True, monitor="testMonitor", units="pix", color='black')
keyb = keyboard.Keyboard()

#Presentation variables
numberSpace = 256
whiteSize = 270
whiteDot = 4
disp = 300
texMod = 462
duration = 600                    #length of each trial, p sure refreshing at 60Hz, 10s trials atm
opac = 1.0
#Trial variables
presNum = np.random.randint(0,2)  #decides if white left or white right happens first
testWL = 0
testWR = 0

#Set up number presentation geometries
texRing = np.random.normal(0, 0.5, (whiteSize, whiteSize))
grayRing = np.ones((numberSpace, numberSpace))
whiteFix = np.ones((whiteDot, whiteDot))

#White Num Stim
texStimR = visual.GratingStim(win=win,tex=texRing,mask="circle",size=(whiteSize, whiteSize),autoDraw=True,pos=[-disp,0])
grayStimR = visual.GratingStim(win=win,tex=grayRing,mask="circle",size=(numberSpace, numberSpace),autoDraw=True,pos=[-disp,0],color=(0,0,0))
whiteNumL = visual.TextStim(win,bold=True,units='pix',pos=[-disp+texMod,0],text=str(np.random.randint(0,10)), color=(1.0,1.0,1.0), opacity=opac, height=90.0)
whiteNumR = visual.TextStim(win,bold=True,units='pix',pos=[disp+texMod,0],text=str(np.random.randint(0,10)), color=(1.0,1.0,1.0), opacity=opac, height=90.0)
fixationR = visual.GratingStim(win=win,tex=whiteFix,mask="circle",size=(whiteDot, whiteDot),autoDraw=True,pos=[-disp,0])

#Black Num Stim
texStimL = visual.GratingStim(win=win,tex=texRing,mask="circle",size=(whiteSize, whiteSize),autoDraw=True,pos=[disp,0])
grayStimL = visual.GratingStim(win=win,tex=grayRing,mask="circle",size=(numberSpace, numberSpace),autoDraw=True,pos=[disp,0],color=(0,0,0))
blackNumR = visual.TextStim(win,bold=True,units='pix',pos=[disp+texMod,0],text=str(np.random.randint(0,10)), color=(-1.0,-1.0,-1.0), opacity=opac, height=90.0)
blackNumL = visual.TextStim(win,bold=True,units='pix',pos=[-disp+texMod,0],text=str(np.random.randint(0,10)), color=(-1.0,-1.0,-1.0), opacity=opac, height=90.0)
fixationB = visual.GratingStim(win=win,tex=whiteFix,mask="circle",size=(whiteDot, whiteDot),autoDraw=True,pos=[disp,0])

while ((testWL < 2) or (testWR < 2)):  #while not w/in a 60/40 time ratio
    whiteRTime = 0
    blackLTime = 0
    whiteLTime = 0
    blackRTime = 0
    timeRatioWL = 0
    timeRatioWR = 0
    if (testWL < 2):       #Only run through if ratios aren't good/strong enough
        if presNum%2 == 0: #White left, Black Right
            #Initialize
            instr = visual.TextStim(win,bold=True,units='pix',pos=[-150+texMod,250],text=str("Align Fixations, Press any key"), color=(1.0,1.0,1.0), opacity=1.0, height=20.0)
            instr.autoDraw = True
            whiteNumL.autoDraw=True
            blackNumR.autoDraw=True
            win.flip()
            core.wait(1) #Prevents subject from hitting key on previous trial to start next one
            event.waitKeys()
            instr.autoDraw = False
            whiteChk = 0 #1 if white was pressed first
            WCount = 0   #Counts num times white pressed
            blackChk = 0 # 1 if black pressed first
            BCount = 0   #Coutns num times b pressed
            
            #Run Trial
            keyb.clock.reset()  # reset keyboard timer
            for i in range(duration):
                keys = keyb.getKeys(['w', 'b'], waitRelease=False)
                if (i%8 == 0):
                    whiteNumL.text = str(np.random.randint(0,10))
                if (i%6 == 0):    
                    blackNumR.text = str(np.random.randint(0,10))
                for key in keys:
                    if key == 'w':
                        whiteLTime += key.rt #Pretty sure this measure keypress onset but not 100% positive
                        WCount+=1  #Counts num times W was pressed
                        keyb.clock.reset()  # reset keyboard timer
                    if key == 'b':
                        blackRTime += key.rt #Also p sure is in seconds, amt time color was white
                        BCount+=1 #Counts num times B was pressed
                        keyb.clock.reset()  # reset keyboard timer
                win.flip()
                if ((whiteLTime == 0) and (blackRTime != 0)):  #Color started white/mix and then switched to black
                    blackChk = 1
                if ((whiteLTime != 0) and (blackRTime == 0)):  #Color started black/mix and then switched to white
                    whiteChk = 1
            #Timing addition for last color seen (no keypress data)
            if ((whiteChk == 1) and (WCount > BCount)):  #White pressed first, color was white when expt ended
                blackRTime += (duration/60-(blackRTime+whiteLTime)) #Add leftover time to BRT, gives more weight to white
            if ((whiteChk == 1) and (WCount == BCount)):  #White pressed first, color was black when expt ended
                whiteLTime += (duration/60-(blackRTime+whiteLTime)) #Add leftover time to WRT, gives more weight to black
            if ((blackChk == 1) and (BCount > WCount)):  #Black pressed first, color was black when expt ended
                whiteLTime += (duration/60-(blackRTime+whiteLTime)) #Add leftover time to WRT, gives more weight to black
            if ((blackChk == 1) and (BCount == WCount)):  #Black pressed first, color was white when expt ended
                blackRTime += (duration/60-(blackRTime+whiteLTime)) #Add leftover time to BRT, gives more weight to black
            #Opacity Ladder (stop after 2 right in a row)
            if blackRTime == 0:
                win.close()
                print('\n \n****You must hit the b and w key during the trial****')
            timeRatioWL = whiteLTime/blackRTime
            if ((timeRatioWL < 3/2) and (timeRatioWL > 2/3)):
                testWL+=1
            elif ((timeRatioWL > 3/2) or (timeRatioWL < 2/3)):
                if (testWL > 0):
                    testWL-=1
                if (whiteLTime > blackRTime):   #Greater white time meand more time seeing black and vice versa
                    if (blackNumR.opacity > 0.1):
                        blackNumR.opacity-=0.1
                    if (whiteNumL.opacity <= 0.9):
                        whiteNumL.opacity+=0.1
                else:
                    if (whiteNumL.opacity > 0.1):
                        whiteNumL.opacity-=0.1
                    if (blackNumR.opacity <= 0.9):
                        blackNumR.opacity+=0.1
    if (testWR < 2):        #Only run through if ratios aren't good/strong enough
        if presNum%2 == 1:  #White right, Black left
            #Initialize
            instr = visual.TextStim(win,bold=True,units='pix',pos=[-150+texMod,250],text=str("Align Fixations, Press any key"), color=(1.0,1.0,1.0), opacity=1.0, height=20.0)
            instr.autoDraw = True
            whiteNumR.autoDraw=True
            blackNumL.autoDraw=True
            win.flip()
            core.wait(1)  #Prevents subject from hitting key on previous trial to start next one
            event.waitKeys()
            instr.autoDraw = False
            whiteChk = 0 #1 if white was pressed first
            WCount = 0   #Counts num times white pressed
            blackChk = 0 # 1 if black pressed first
            BCount = 0   #Coutns num times b pressed
            
            #Run Trial
            keyb.clock.reset()  # reset keyboard timer 
            for i in range(duration):
                keys = keyb.getKeys(['w', 'b'], waitRelease=False)
                if (i%8 == 0):
                    whiteNumR.text = str(np.random.randint(0,10))
                if (i%6 == 0):    
                    blackNumL.text = str(np.random.randint(0,10))
                for key in keys:
                    if key == 'w':
                        whiteRTime+=key.rt #Pretty sure this measure keypress onset but not 100% positive
                        WCount+=1  #Counts num times W was pressed
                        keyb.clock.reset()  # reset keyboard timer
                    if key == 'b':
                        blackLTime+=key.rt #Also p sure is in seconds
                        BCount+=1  #Counts num times B was pressed
                        keyb.clock.reset()  # reset keyboard timer
                win.flip()
                if ((whiteRTime == 0) and (blackLTime != 0)):  #Color started white/mix and then switched to black
                    blackChk = 1
                if ((whiteRTime != 0) and (blackLTime == 0)):  #Color started black/mix and then switched to white
                    whiteChk = 1
            #Timing addition for last color seen (no keypress data)
            if ((whiteChk == 1) and (WCount > BCount)):  #White pressed first, color was white when expt ended
                blackLTime += (duration/60-(blackLTime+whiteRTime)) #Add leftover time to BRT, gives more weight to white
            if ((whiteChk == 1) and (WCount == BCount)):  #White pressed first, color was black when expt ended
                whiteRTime += (duration/60-(blackLTime+whiteRTime)) #Add leftover time to WRT, gives more weight to black
            if ((blackChk == 1) and (BCount > WCount)):  #Black pressed first, color was black when expt ended
                whiteRTime += (duration/60-(blackLTime+whiteRTime)) #Add leftover time to WRT, gives more weight to black
            if ((blackChk == 1) and (BCount == WCount)):  #Black pressed first, color was white when expt ended
                blackLTime += (duration/60-(blackLTime+whiteRTime)) #Add leftover time to BRT, gives more weight to black
            #Opacity Ladder (stop after 2 right in a row)
            if blackLTime == 0:
                win.close()
                print('\n \n****You must hit the b and w key during the trial****')
            timeRatioWR = whiteRTime/blackLTime
            if ((timeRatioWR < 3/2) and (timeRatioWR > 2/3)):
                testWR+=1
            elif ((timeRatioWR > 3/2) or (timeRatioWR < 2/3)):
                if (testWR > 0):
                    testWR-=1
                if (whiteRTime > blackLTime):   #Greater white time means more time seeing black and vice versa
                    if (blackNumL.opacity > 0.1):
                        blackNumL.opacity-=0.1
                    if (whiteNumR.opacity <= 0.9):
                        whiteNumR.opacity+=0.1
                else:
                    if (whiteNumR.opacity > 0.1):
                        whiteNumR.opacity-=0.1
                    if (blackNumL.opacity <= 0.9):
                        blackNumL.opacity+=0.            

    whiteNumL.autoDraw=False
    blackNumR.autoDraw=False
    whiteNumR.autoDraw=False
    blackNumL.autoDraw=False
    presNum+=1
        
win.close()
print("WR opac =", whiteNumR.opacity, "BL opac =", blackNumL.opacity, "WL opac =", whiteNumL.opacity, "BR opac =", blackNumR.opacity)
#Note on opacity Ladder: whiteRT is effectively the amt time the subject saw black (b/c hitting the w key indicates
#                        a shift to the white color from the black color) and vice versa.
#                        If whiteTime>blackTime, then the white intensity up and black down